In [ ]:
!pip install pytorch_lightning

In [ ]:
#import all the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, mean_squared_error
import pandas as pd
import os
import math
import matplotlib.pyplot as plt

In [ ]:
input_path = '/content/drive/My Drive/AMLProject/Data/2024-flame-ai-challenge/dataset'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#the idea and part of the code for positional and spatial encoding waas taken from the second best solution
# in the kaggle competition - https://www.kaggle.com/competitions/2024-flame-ai-challenge/discussion/541458
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=150):
        super().__init__()
        position = torch.arange(max_seq_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_seq_length, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0)]

In [ ]:
class SpatialEncoding(nn.Module):
    def __init__(self, d_model, height=113, width=32):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(d_model, height, width)
        for pos_h in range(height):
            for pos_w in range(width):
                for i in range(0, d_model, 2):
                    pe[i, pos_h, pos_w] = math.sin(pos_h / (10000 ** (i / d_model)))
                    pe[i + 1, pos_h, pos_w] = math.cos(pos_w / (10000 ** ((i + 1) / d_model)))
        self.register_buffer('pe', pe)
            # Initialize loss tracking
        self.epoch_train_losses = []
        self.epoch_val_losses = []
        self.validation_step_outputs = []

    def forward(self, x):
        return x + self.pe

In [ ]:
class PredFormer(pl.LightningModule):
    def __init__(self, input_dim=7, d_model=32, nhead=8, num_layers=4):
        super().__init__()
        self.input_dim = input_dim
        self.d_model = d_model

        # Input embedding
        self.input_proj = nn.Conv2d(input_dim, d_model, 3, padding=1)
        self.velocity_proj = nn.Conv2d(d_model, 2, 1)

        # Positional encodings
        self.pos_encoder = PositionalEncoding(d_model)
        self.spatial_encoder = SpatialEncoding(d_model)

        # Transformer layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=1024,
            batch_first=True,
            dropout=0.1
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)

        # Output projection
        self.output_proj = nn.Conv2d(d_model, 1, 3, padding=1)
        self.residual_conv = nn.Conv2d(input_dim, 1, 1)

        # Metrics tracking
        self.train_metrics = []
        self.val_metrics = []

    def create_flow_grid(self, flow):
        b, _, h, w = flow.size()
        device = flow.device
        grid_x = torch.linspace(-1, 1, w, device=device)
        grid_y = torch.linspace(-1, 1, h, device=device)
        grid_y, grid_x = torch.meshgrid(grid_y, grid_x)
        grid = torch.stack([grid_x, grid_y], dim=-1)
        grid = grid.unsqueeze(0).expand(b, -1, -1, -1)
        flow = flow.permute(0, 2, 3, 1)
        grid = grid + flow
        return torch.clamp(grid, -1, 1)

    def forward(self, x, future_steps):
        b, t, c, h, w = x.size()
        x_proj = x.view(b * t, c, h, w)
        features = self.input_proj(x_proj)
        velocity = self.velocity_proj(features).view(b, t, 2, h, w)

        features = features.view(t, b, self.d_model, h, w)
        features = self.spatial_encoder(features)
        features = features.permute(0, 1, 3, 4, 2).reshape(t, b * h * w, self.d_model)
        features = self.pos_encoder(features)

        memory = self.transformer(features)
        outputs = []
        current_state = x[:, -1, -1]
        current_velocity = velocity[:, -1]

        for _ in range(future_steps):
            feature_pred = memory[-1:].reshape(1, b, h, w, self.d_model)
            feature_pred = feature_pred.permute(0, 1, 4, 2, 3)
            out = self.output_proj(feature_pred.reshape(-1, self.d_model, h, w))
            residual = self.residual_conv(x[:, -1])

            grid = self.create_flow_grid(current_velocity)
            moved_state = F.grid_sample(
                current_state.unsqueeze(1),
                grid,
                mode='bilinear',
                padding_mode='zeros',
                align_corners=True
            )

            final_pred = out + residual + moved_state
            outputs.append(final_pred)
            current_state = final_pred.squeeze(1)
            memory = torch.cat([memory[1:], memory[-1:]], 0)

        return torch.stack(outputs, dim=1).squeeze(2)

    def training_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self(x, future_steps=y.size(1))

      # Calculate losses
      mse_loss = F.mse_loss(y_hat, y)
      movement_loss = F.mse_loss(y_hat[:, 1:] - y_hat[:, :-1], y[:, 1:] - y[:, :-1])
      loss = mse_loss + 0.5 * movement_loss

     # Calculate metrics - detach tensors before converting to numpy
      y_flat = y.reshape(-1).cpu().detach()
      y_hat_flat = y_hat.reshape(-1).cpu().detach()
      f1 = f1_score((y_flat > 0.5).numpy(), (y_hat_flat > 0.5).numpy(), average='binary')
      roc_auc = roc_auc_score((y_flat > 0.5).numpy(), y_hat_flat.numpy())

      self.log_dict({
          'train_loss': loss,
          'train_f1': f1,
          'train_roc_auc': roc_auc
      })
      return loss

    def validation_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self(x, future_steps=y.size(1))

      # Calculate losses
      mse_loss = F.mse_loss(y_hat, y)
      movement_loss = F.mse_loss(y_hat[:, 1:] - y_hat[:, :-1], y[:, 1:] - y[:, :-1])
      loss = mse_loss + 0.5 * movement_loss

      # Calculate metrics - detach tensors before converting to numpy
      y_flat = y.reshape(-1).cpu().detach()
      y_hat_flat = y_hat.reshape(-1).cpu().detach()
      f1 = f1_score((y_flat > 0.5).numpy(), (y_hat_flat > 0.5).numpy(), average='binary')
      roc_auc = roc_auc_score((y_flat > 0.5).numpy(), y_hat_flat.numpy())

      self.log_dict({
          'val_loss': loss,
          'val_f1': f1,
          'val_roc_auc': roc_auc
      })
      return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss"
        }

In [ ]:
class WildfireDataset(Dataset):
    def __init__(self, csv_file, data_dir, sequence_length=5, prediction_length=20):
        self.data = pd.read_csv(csv_file)
        self.data_dir = data_dir
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        self.max_u = self.data['u'].max()
        self.max_alpha = self.data['alpha'].max()

        # Calculate total number of sequences
        self.sequences_per_file = 150 - sequence_length - prediction_length + 1
        self.total_sequences = len(self.data) * self.sequences_per_file

    def __len__(self):
        return self.total_sequences

    def __getitem__(self, idx):
        # Calculate which file and which sequence within the file
        file_idx = idx // self.sequences_per_file
        sequence_start = idx % self.sequences_per_file

        row = self.data.iloc[file_idx]

        # Load data
        ustar = np.fromfile(f"{self.data_dir+'/train'}/{row['ustar_filename']}", dtype=np.float32).reshape(row['Nt'], row['Nx'], row['Ny'])
        theta = np.fromfile(f"{self.data_dir+'/train'}/{row['theta_filename']}", dtype=np.float32).reshape(row['Nt'], row['Nx'], row['Ny'])
        xi = np.fromfile(f"{self.data_dir+'/train'}/{row['xi_filename']}", dtype=np.float32).reshape(row['Nt'], row['Nx'], row['Ny'])

        # Create global parameter channels
        u_channel = np.full((row['Nt'], 1, row['Nx'], row['Ny']), row['u'], dtype=np.float32)
        alpha_channel = np.full((row['Nt'], 1, row['Nx'], row['Ny']), row['alpha'], dtype=np.float32)

        # Normalize data
        # Add safety check for normalization
        if np.std(ustar) == 0:
          ustar = ustar - np.mean(ustar)  # Only center if std is zero
        else:
          ustar = (ustar - np.mean(ustar)) / np.std(ustar)
        if np.std(theta) == 0:
          theta = theta - np.mean(theta)  # Only center if std is zero
        else:
          theta = (theta - np.mean(theta)) / np.std(theta)

        # Normalize global parameters
        u_channel = u_channel / self.max_u
        alpha_channel = alpha_channel / self.max_alpha

        # Select sequence window
        sequence_end = sequence_start + self.sequence_length
        target_start = sequence_end
        target_end = target_start + self.prediction_length

        # Create input sequence
        ustar_seq = ustar[sequence_start:sequence_end]
        theta_seq = theta[sequence_start:sequence_end]
        xi_seq = xi[sequence_start:sequence_end]
        u_seq = u_channel[sequence_start:sequence_end]
        alpha_seq = alpha_channel[sequence_start:sequence_end]

        # Stack input channels
        input_seq = np.stack([ustar_seq, theta_seq, xi_seq], axis=1)

        # Add positional encoding
        pos_enc = self.positional_encoding(self.sequence_length, row['Nx'], row['Ny'])

        # Concatenate all channels
        input_seq = np.concatenate([input_seq, pos_enc, u_seq, alpha_seq], axis=1)

        # Create target sequence
        if target_end <= 150:
            target = xi[target_start:target_end]
        else:
            # Pad with zeros if target extends beyond available timesteps
            available_steps = 150 - target_start
            target = np.zeros((self.prediction_length, row['Nx'], row['Ny']), dtype=np.float32)
            if available_steps > 0:
                target[:available_steps] = xi[target_start:150]

        return torch.FloatTensor(input_seq), torch.FloatTensor(target)

    def positional_encoding(self, seq_len, height, width):
        pos_enc = np.zeros((seq_len, 2, height, width), dtype=np.float32)
        for t in range(seq_len):
            for i in range(height):
                for j in range(width):
                    pos_enc[t, 0, i, j] = i / height
                    pos_enc[t, 1, i, j] = j / width
        return pos_enc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv(os.path.join(input_path, 'train.csv'))

In [ ]:
# 7. Initialize dataset and create data loaders
train_dataset = WildfireDataset(
    csv_file=os.path.join(input_path, 'train.csv'),
    data_dir=input_path,
    sequence_length=5,
    prediction_length=20
)

# 8. Create train-validation split and data loaders
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(
    train_subset,
    batch_size=32,
    shuffle=True,
    num_workers=2,  # Increase from 4
    pin_memory=True  # Enable for GPU training
)
val_loader = DataLoader(
    val_subset,
    batch_size=32,
    shuffle=False,  # Set to False for validation
    num_workers=2,
    pin_memory=True
)


In [ ]:
torch.cuda.empty_cache()  # Clear GPU memory before training
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
# Add to environment variables
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear cache before training
torch.cuda.empty_cache()

In [ ]:
model = PredFormer(
    input_dim=7,      # Keep input channels
    d_model=32,
    nhead=4,
    num_layers=2
)
#model = torch.compile(model)
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(output_path, 'checkpoints'),
    filename='wildfire-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    monitor='val_loss',
    mode='min'
)

torch.cuda.empty_cache()  # Clear GPU memory

trainer = pl.Trainer(
    max_epochs=20,
    accelerator='cpu',  # Change from 'gpu' to 'cpu'
    callbacks=[checkpoint_callback],
    default_root_dir=output_path,
    gradient_clip_val=0.5,
    accumulate_grad_batches=4
)

trainer.fit(model, train_loader, val_loader)

def plot_training_history(model):
    plt.figure(figsize=(10, 6))
    plt.plot(model.epoch_train_losses, label='Training Loss')
    plt.plot(model.epoch_val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.grid(True)
    plt.show()

# 10. Start training
plot_training_history(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/My Drive/AMLProject/Data/2024-flame-ai-challenge/checkpoints exists and is not empty.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | input_proj      | Conv2d             | 2.0 K  | train
1 | velocity_proj   | Conv2d             | 66     | train
2 | pos_encoder     | PositionalEncoding | 0      | train
3 | spatial_encoder | SpatialEncoding    | 0      | train
4 | transformer     | TransformerEncoder | 141 K  | train
5 | output_proj     | Conv2d             | 289    | train
6 | residual_conv   | Conv2

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import torch
from google.colab import files

def create_and_save_fire_animation(model, test_loader, filename=None):
    x_test, y_test = next(iter(test_loader))
    x = x_test[0:1]
    y_true = y_test[0]

    with torch.no_grad():
        y_pred = model(x, future_steps=y_true.size(0))
        y_pred = y_pred[0]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Create imshow objects once
    im1 = ax1.imshow(y_true[0].cpu(), cmap='hot', vmin=0, vmax=1)
    im2 = ax2.imshow(y_pred[0].cpu(), cmap='hot', vmin=0, vmax=1)

    # Create colorbars once
    fig.colorbar(im1, ax=ax1)
    fig.colorbar(im2, ax=ax2)

    def update(frame):
        im1.set_array(y_true[frame].cpu())
        im2.set_array(y_pred[frame].cpu())
        ax1.set_title(f'Actual Fire (t={frame})')
        ax2.set_title(f'Predicted Fire (t={frame})')
        return im1, im2

    anim = animation.FuncAnimation(
        fig, update, frames=y_true.size(0),
        interval=200, blit=True
    )

    if filename:
        if filename.endswith('.mp4'):
            Writer = animation.writers['ffmpeg']
            writer = Writer(fps=5, metadata=dict(artist='Me'), bitrate=1800)
            anim.save(filename, writer=writer)
            files.download(filename)
        elif filename.endswith('.gif'):
            anim.save(filename, writer='pillow')
            files.download(filename)

    plt.close()
    return anim

# Create and save animations
anim_gif = create_and_save_fire_animation(model, val_loader, 'fire_prediction.gif')

# Display in notebook
HTML(anim_gif.to_jshtml())